In [31]:
import pandas as pd

In [32]:
#install genism package for word2vec
!pip3 install --upgrade gensim

In [3]:
!pip3 install transformers

In [4]:
!pip3 install nltk

In [5]:
!pip3 install torch

In [6]:
!pip3 install matplotlib

In [7]:
import os
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.tag import pos_tag
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#import mxnet as mx
#from bert_embedding import BertEmbedding
#from sentence_transformers import SentenceTransformer
import gensim 
from gensim.models import Word2Vec
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to C:\Users\Loke
[nltk_data]     Yi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Loke
[nltk_data]     Yi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Loke Yi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Loke
[nltk_data]     Yi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Loke Yi\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g.


## **Import data and spilt it into train and test datasets**


In [8]:
#train datafile 
review_data = pd.read_csv("Shopee Reviews Data/train.csv", index_col=0)
review_data.rename(columns = {'rating': 'label'}, inplace = True)
review_data = review_data.sample(frac=1)
review_data.head()

,review,label
review_id,,
74686,Awesome awesome quality of goods shipped speed,4
137337,Baguussss .. Like,5
35924,Product quality good price good product,3
66144,Until the appropriate time of booking. Sampeny...,4
100248,Pengiriimann very cepattt.,4


In [9]:
review_data.reset_index(drop=True, inplace=True)
review_data.head()

,review,label
0,Awesome awesome quality of goods shipped speed,4
1,Baguussss .. Like,5
2,Product quality good price good product,3
3,Until the appropriate time of booking. Sampeny...,4
4,Pengiriimann very cepattt.,4


In [10]:
review_data.describe()

,label
count,146811.000000
mean,3.562764
std,1.260537
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [11]:
#test datafile 
#test_data = pd.read_csv("Shopee Reviews Data/test.csv", index_col=0)
#test_data = test_data.sample(frac=1)
#test_data.head()

In [12]:
#test_data.describe()

In [13]:
#label
y= review_data.label
#features
x=review_data.drop('label', axis=1)

In [14]:
#split into test and train dataset with test size 20% 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
x_train.head()

,review
136284,Good value CP bad bad service bad bad bad bad...
94586,"Pretty lah, small mouth interchangeable 2 😃tin..."
57477,Product quality good
121853,"The response was a bit slow seller, ask for ad..."
49041,packing an good !!


In [15]:
x_test.head()

,review
143983,The product quality is excellent.
51264,"Alhamdulillah already come its neat packaging,..."
136746,"Pesen 24th, recently sent date 27. Sellers nev..."
21248,"Jazakillah Khairan katsiran. Cepet really, che..."
140311,The product quality is excellent.


In [16]:
x_train.describe()

,review
count,117448
unique,92568
top,Excellent product quality
freq,1098


In [17]:
x_test.describe()

,review
count,29363
unique,23928
top,Excellent product quality
freq,280


## **Text-preprocessing** 

In [18]:
def NormalizeWithPOS(text):
    # Lemmatization & Stemming according to POS tagging

    #tokenize snetence into a list of single words 
    word_list = word_tokenize(text)
    rev = []
    
    lemmatizer = WordNetLemmatizer() 
    stemmer = PorterStemmer() 
    
    #pos_tag() get the POS tag of given word 
    #'J'--> Adjective 
    #'V'--> Verb
    #'N'--> Noun
    #'R'--> Adverb 
    
    for word, tag in pos_tag(word_list):
        if tag.startswith('J'):
            w = lemmatizer.lemmatize(word, pos='a')
        elif tag.startswith('V'):
            w = lemmatizer.lemmatize(word, pos='v')
        elif tag.startswith('N'):
            w = lemmatizer.lemmatize(word, pos='n')
        elif tag.startswith('R'):
            w = lemmatizer.lemmatize(word, pos='r')
        else:
            w = word
        w = stemmer.stem(w)
        rev.append(w)
    review = ' '.join(rev)
    return review

In [19]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
def cleanText(text):
    
    #replace some specific words 
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"[0-9]+", ' ', text)
    text = re.sub(r"-", ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.strip().lower()

    #remove stop words 
    stop_words=set(stopwords.words('english'))
    word_tokens=word_tokenize(text)
    filtered_list=[w for w in word_tokens if not w in stop_words]
    text=' '.join(filtered_list)

    # Replace punctuations with space
    if embedding is 'BERT': # save ! ? . for end of the sentence detection [,/():;']
        filters='"#$%&*+<=>@[\\]^_`{|}~\t\n'
        text = re.sub(r'\!+', '!', text)
        text = re.sub(r'\?+', '?', text)
    else:
        filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    if embedding is 'BERT':
        text = re.sub(r'\( *\)', ' ', text)

    if embedding is not 'BERT':
        text = ' '.join([w for w in text.split() if len(w)>1])

    text = ''.join(text)

    return text

In [21]:
embedding=''
x_train['clean_text']= x_train['review'].apply(cleanText)
x_test['clean_text']=x_test['review'].apply(cleanText)

## **Visual comparision of clean and before clean text** 

In [22]:
print("A review example of dataset before cleaning:", end="\n\n")
print(x_train.iloc[1112]['review'], end='\n\n')


print("clean_text:")
print(x_train.iloc[1112]['clean_text'], end="\n\n")

A review example of dataset before cleaning:

Semoooogggaaaaaa cocooooooooooooookkkkkkkkkkkkkkkkkk

clean_text:
semoooogggaaaaaa cocooooooooooooookkkkkkkkkkkkkkkkkk



In [23]:
#normalized with POS 
x_train['clean_text_normalized']=x_train['clean_text'].apply(NormalizeWithPOS)
x_test['clean_text_normalized']=x_test['clean_text'].apply(NormalizeWithPOS)

In [24]:
print("A review example of dataset before cleaning:", end="\n\n")
print(x_train.iloc[1114]['review'], end='\n\n')


print("clean_text_normalized:")
print(x_train.iloc[1114]['clean_text_normalized'], end="\n\n")

A review example of dataset before cleaning:

 Excellent product quality product packaging is very beautiful and certainly very fast delivery time. Thank shop sir ❤

clean_text_normalized:
excel product qualiti product packag beauti certainli fast deliveri time thank shop sir



## **Low frequency words of train data**
for BOW and TFIDF


In [25]:
freq_train= pd.Series(' '.join(x_train['clean_text_normalized']).split()).value_counts()
less_than_five_freq_train= freq_train[(freq_train<5)]
print(less_than_five_freq_train)

dumatng       4
realist       4
builder       4
flaki         4
defeat        4
             ..
teksturnya    1
mksimal       1
dof           1
kemasannya    1
siku🤣         1
Length: 42964, dtype: int64


## **Low frequency words of test data**
for BOW and TFIDF


In [26]:
freq_test= pd.Series(' '.join(x_test['clean_text_normalized']).split()).value_counts()
less_than_five_freq_test= freq_test[(freq_test<5)]
print(less_than_five_freq_test)

hinh        4
wlpn        4
😄😄😄         4
lagiiii     4
io          4
           ..
discreet    1
nitto       1
😍😍😍👍👍👍      1
exposur     1
habiss      1
Length: 17141, dtype: int64


## **Remove low frequency words which less than 5** 

In [27]:
x_train['clean_text_normalized']= x_train['clean_text_normalized'].apply(lambda x: ' '
.join(x for x in x.split() if x not in less_than_five_freq_train))
x_test['clean_text_normalized']=x_test['clean_text_normalized'].apply(lambda x: ' '
.join(x for x in x.split() if x not in less_than_five_freq_test))


In [28]:
embedding= 'BOW'
vectorizer= CountVectorizer(stop_words='english')

bow_training_features= vectorizer.fit_transform(x_train['clean_text_normalized'])
bow_test_features= vectorizer.transform(x_test['clean_text_normalized'])

print(bow_training_features.shape)

(117448, 7935)


In [35]:
# A function to plot and print result
def printResult(y_pred, y_prob):
    acc = accuracy_score(y_test, y_pred)
    # Result
    print("Accuracy: {:.2f}".format(acc*100),end='\n\n')
    cm = confusion_matrix (y_test,y_pred)
    print('Confusion Matrix:\n', cm)
    print(classification_report(y_test,y_pred))

In [30]:
from sklearn.svm import LinearSVC
model = LinearSVC(random_state=0, tol=1e-5)
#model = SVC(kernel ='linear', C = 1)

In [33]:
# Training 
model.fit(bow_training_features, y_train)

# Evaluation
y_pred_bow_svm = model.predict(bow_test_features)
y_prob_bow_svm = model.decision_function(bow_test_features)

In [36]:
# Result
printResult(y_pred_bow_svm, y_prob_bow_svm)

Accuracy: 43.82

Confusion Matrix:
 [[1597  194  811  169  173]
 [ 525  546 1058  247  226]
 [ 462  307 4406 1059  973]
 [ 164   84 1733 3508 2890]
 [ 183   93 1634 3511 2810]]
              precision    recall  f1-score   support

           1       0.54      0.54      0.54      2944
           2       0.45      0.21      0.29      2602
           3       0.46      0.61      0.52      7207
           4       0.41      0.42      0.42      8379
           5       0.40      0.34      0.37      8231

    accuracy                           0.44     29363
   macro avg       0.45      0.42      0.43     29363
weighted avg       0.44      0.44      0.43     29363



In [37]:
embedding = 'TFIDF_WITH_STOP'
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=20000)

tfidf_with_stop_training_features = vectorizer.fit_transform(x_train["clean_text_normalized"])    
tfidf_with_stop_test_features = vectorizer.transform(x_test["clean_text_normalized"])

print(tfidf_with_stop_training_features.shape)

(117448, 20000)


In [38]:
# Training 
model.fit(tfidf_with_stop_training_features, y_train)

# Evaluation
y_pred_tfidf_svm = model.predict(tfidf_with_stop_test_features)
y_prob_tfidf_svm = model.decision_function(tfidf_with_stop_test_features)

In [39]:
# Result
printResult(y_pred_tfidf_svm, y_prob_tfidf_svm)

Accuracy: 45.13

Confusion Matrix:
 [[1773  324  509  171  167]
 [ 593  692  862  219  236]
 [ 473  465 4131 1043 1095]
 [ 186  121 1334 3271 3467]
 [ 183  114 1264 3286 3384]]
              precision    recall  f1-score   support

           1       0.55      0.60      0.58      2944
           2       0.40      0.27      0.32      2602
           3       0.51      0.57      0.54      7207
           4       0.41      0.39      0.40      8379
           5       0.41      0.41      0.41      8231

    accuracy                           0.45     29363
   macro avg       0.46      0.45      0.45     29363
weighted avg       0.45      0.45      0.45     29363



In [45]:
model = MultinomialNB()

In [46]:
# Training 
model.fit(bow_training_features.toarray(), y_train)

# Evaluation
y_pred_bow_nb = model.predict(bow_test_features.toarray())
y_prob_bow_nb = model.predict_proba(bow_test_features.toarray())[:,1]

MemoryError: Unable to allocate 6.94 GiB for an array with shape (117448, 7935) and data type int64

In [ ]:
# Result
printResult(y_pred_bow_nb, y_prob_bow_nb)